## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-15-21-48-56 +0000,nypost,White House investigating Biden use of autopen...,https://nypost.com/2025/07/15/us-news/white-ho...
1,2025-07-15-21-44-50 +0000,bbc,Israel bombs Syrian forces entering Druze city...,https://www.bbc.com/news/articles/c89e3j4e911o
2,2025-07-15-21-42-57 +0000,bbc,"Reeves inadvertently breached rules on gifts, ...",https://www.bbc.com/news/articles/c89e2lpx5eeo
3,2025-07-15-21-39-00 +0000,wsj,Labor Department Relied More on Fuzzier Price ...,https://www.wsj.com/economy/labor-department-r...
4,2025-07-15-21-36-59 +0000,bbc,Man who murdered British backpacker Peter Falc...,https://www.bbc.com/news/articles/cjrlg4x8e78o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2351/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,23
170,ukraine,8
70,tariffs,7
67,china,6
47,new,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
91,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,49
35,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,47
76,2025-07-15-09-38-38 +0000,bbc,'Not our war' - Trump's Nato weapons deal for ...,https://www.bbc.com/news/articles/c14e2ydv4d6o,44
56,2025-07-15-13-32-31 +0000,nyt,China’s Economy Grows Steadily Despite Trump’s...,https://www.nytimes.com/2025/07/14/business/ch...,41
14,2025-07-15-20-35-44 +0000,nypost,House Speaker Mike Johnson breaks with Trump o...,https://nypost.com/2025/07/15/us-news/house-sp...,41


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
91,49,2025-07-15-02-01-34 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o
13,26,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
15,24,2025-07-15-20-32-50 +0000,nypost,NYC photog captures stunning video of lightnin...,https://nypost.com/2025/07/15/us-news/nyc-phot...
35,24,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
39,19,2025-07-15-18-44-15 +0000,nypost,Mayor Eric Adams finally gets good news in lon...,https://nypost.com/2025/07/15/us-news/mayor-ad...
14,18,2025-07-15-20-35-44 +0000,nypost,House Speaker Mike Johnson breaks with Trump o...,https://nypost.com/2025/07/15/us-news/house-sp...
44,18,2025-07-15-17-14-09 +0000,bbc,Thousands of Afghans were moved to UK in secre...,https://www.bbc.com/news/articles/cvg8zy78787o
82,17,2025-07-15-06-42-06 +0000,bbc,"As theories swirl about Air India crash, key d...",https://www.bbc.com/news/articles/cp3lpdqp7x3o
6,16,2025-07-15-21-28-25 +0000,nypost,Sickening new charges for LI man accused of ra...,https://nypost.com/2025/07/15/us-news/sickenin...
28,16,2025-07-15-19-40-33 +0000,nypost,Diddy joins jail self-improvement programs to ...,https://nypost.com/2025/07/15/us-news/diddy-jo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
